## dekoratory

- umoznuji rozsirovat opakovany zpusobem funkce
- dekorator ma jako vstup funkci a vraci funkci. Obali jeji prubeh do dalsiho kodu
- muzeme vyrabet ruzne dekoratory v zavislosti na dalsich parametrech. Je to docela uzitecne, celkem neprehledne a hlavne se tomu rika "metaprogramovani"

In [ ]:
def dec(func):
    def wrapper(*args, **kwargs):
        print("calling function", func.__name__)
        return func(*args, **kwargs)
    return wrapper

def add(x, y):
    return x + y

decorated = dec(add)

decorated(1,2)

@dec
def subtract(x, y):
    return x - y

subtract(2, 1)

In [ ]:
LOG_INFO    = 0
LOG_WARNING = 1
LOG_DEBUG   = 2

LOG_STR_LST = ["INFO", "WARNING", "DEBUG"]

log_level = LOG_DEBUG

def log(level = LOG_INFO):
    def dec(func):
        def wrapper(*args, **kwargs):
            if level <= log_level:
                print("{}: running function: {}".format(LOG_STR_LST[level], func.__name__))
                if log_level >= LOG_DEBUG:
                    print("\targs:", args)
                    print("\tkwargs:", kwargs)
            return func(*args, **kwargs)
        return wrapper
    return dec

@log(LOG_INFO)
def add(x, y):
    return x + y

@log(LOG_WARNING)
def do_warning_level_stuff():
    pass

@log(LOG_DEBUG)
def do_debug_level_stuff(**kwargs):
    pass

do_debug_level_stuff(neco = True)
add(1, 2)
do_warning_level_stuff()

## Had

In [ ]:
class Snake:
    length = 2
    
    def eat(self):
        Snake.length += 1

had = Snake()
had.eat()
had.eat()

print(had.length)

In [ ]:
had2 = Snake()
print(had2.length)

## class vs instance attributes

- length v prikladu vyse je "class attribute" - promenna tridy, ktera je pristupna bez konkretni instance

In [ ]:
Snake.length = 1
print(had.length)
print(had2.length)


lepsi implementace:

In [ ]:
class Snake:
    start_length = 2
    
    def __init__(self):
        self.length = Snake.start_length
        
    def eat(self):
        self.length += 1

had = Snake()
had.eat()
had.eat()

print(had.length)
Snake.start_length = 1
had2 = Snake()
print(had2.length)

In [ ]:
class Snake:
    start_length = 2
    
    def __init__(self, length = Snake.start_length):
        self.length = length
        
    def eat(self):
        self.length += 1
        
had = Snake(3)
print(had.length)


## magic methods (dunder methods)

In [ ]:
class Snake:
    """
    Thisss isss a docssstring
    """
    start_length = 2
    
    def __init__(self, length = None):
        self.length = length if length is not None else self.start_length
        
    def eat(self):
        self.length += 1
    
    def __len__(self):
        return self.length
    
    def __add__(self, x):
        self.length += x
        return self
    
    def __iadd__(self, x):
        self.length += x
        return self
    
had = Snake()
had.eat()
print(len(had))
had = had + 2
print(len(had))
had +=1
print(len(had))

In [ ]:
dir(Snake)
for key, val in Snake.__dict__.items():
    print(key, val)
    
doc(Snake)

In [ ]:
class Polynomial:
    """Returns a callable Polynomial object."""
    def __init__(self, *coefs):
        self.coefs = coefs
        
    def __call__(self, x):
        val = self.coefs[-1]
        for c in reversed(self.coefs[:-1]):
            val = val * x + c
        return val
    
    def order(self):
        return len(self.coefs - 1)


## `public` vs. `private`
```cpp
class Pizza {
    public:
        double hmotnost;
        string druh;
    private:
        string tajna_prisada;
        int kalorie;
};

pizza  = Pizza();

printf("%s\n", pizza.druh); // ok
printf("%s\n", pizza.tajna_prisada); // big no no ~ Cannot access private member
```


In [ ]:
class Pizza:
    def __init__(self):
        self.hmotnost = 200
        self.druh = "margherita"
        self.__tajna_prisada = "velrybi tuk"
        self._kalorie = 24000 # kcal/m^2
        
    def get_secret(self):
        return self.__tajna_prisada

pizza = Pizza()

# pizza.__tajna_prisada
pizza._Pizza__tajna_prisada
print(pizza.get_secret())

- Python private a public nerozlisuje, vse je public
> *We are all consenting adults. (Anyone can touch your privates.)*


## class vs instance methods

In [ ]:
class Pizza():
    testo = {
        "mouka" : [300, "g"],
        "drozdi" : [5, "g"],
        "voda" : [150, "ml"]
    }
    # instance method
    def __init__(self, druh):
        self.hmotnost = 200
        self.druh = druh
        self._tajna_prisada = "velrybi tuk"
        self._kalorie = 24000 # kcal/m^2
        
    def co_je_to_za_pizzu(self):
        print("tuto je", self.druh)
    
    @classmethod # dekorator
    def ukaz_testo(cls):
        print("Prisady:")
        for key, val in cls.testo.items():
            print("{}: {} {}".format(key, val[0], val[1]))
    
    @staticmethod # dekorator
    def co_to_je_pizza():
        print("pizza je kdyz...")

pizza = Pizza("margherita")
pizza.co_je_to_za_pizzu()
Pizza.ukaz_testo()
Pizza.co_to_je_pizza()

## Iteratory a generatory

In [ ]:
cisla = [1, 2, 3, 4]

for i in cisla:
    print(i)
    
neco = iter(cisla)
print(neco, type(neco))

print(next(neco))
print(next(neco))
print(next(neco))
print(next(neco))
print(next(neco))

In [ ]:
# vlastni iterator
class Iterator:
    def __init__(self, data):
        self.data = data
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == len(self.data):
            raise StopIteration
        self.index += 1
        return self.data[self.index - 1]
    
test = Iterator([1,2,3,5,6])

for i in test:
    print(i)

In [ ]:
class Fibonacci:
    def __init__(self, n=10):
        self.curr = 1
        self.last = 0
        self.it = 1
        self.n = n
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.it > self.n:
            raise StopIteration
        self.it += 1
        
        ret = self.last
        self.last, self.curr = self.curr, self.curr + self.last
        return ret

for fib in Fibonacci(4):
    print(fib)
    

In [ ]:
def fib(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

print(type(fib(10)))
fib(10)

## Vlastní implementace komplexních čísel
Komplexní čísla mají dvě složky: reálnou a imaginární. Jejich aritmetika je odvozena od definující vlastnosti imaginární jednotky $i$, tedy
\begin{align}
    i^2 &= -1\\
    (a + bi) \pm (c + di) &= (a \pm c) + (b\pm d)i\\
    (a + bi) \cdot (c + di) &= (ac - bd) + (ad + bc)i\\
    \frac{(a + bi)}{(c + di)} &= \frac{(ac + bd) + (bc - ad)i}{c^2+d^2}\\
\end{align}
Kromě toho se ještě zavádí komplexně sdružené číslo a absolutní hodnota komplexního čísla:
\begin{align}
    (a + bi)^* &= a - bi\\
    |a+bi| &= \sqrt{a^2 + b^2}
\end{align}

In [ ]:
from math import sqrt
class Complex:
    """A simple implementation of the complex type. Division not yet implemented."""
    # TODO: implement division
    print_prec = 2
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag
    
    def __abs__(self):
        return sqrt(self.real**2 + self.imag**2)
    
    def conj(self):
        return Complex(self.real, -self.imag)
    
    def __eq__(self, other):
        if (self.real == other.real) and (self.imag == other.imag):
            return True
        return False
    
    def __neq__(self, other):
        return not self == other
    
    def __add__(self, other):
        if type(other) in [float, int]:
            return Complex(self.real + other, self.imag)
        else:
            return Complex(self.real + other.real, self.imag + other.imag)
        
    def __radd__(self, other):
        return Complex.__add__(self, other)
    
    def __sub__(self, other):
        if type(other) in [float, int]:
            return Complex(self.real - other, self.imag)
        else:
            return Complex(self.real - other.real, self.imag - other.imag)
        
    def __rsub__(self, other):
        return Complex.__sub__(other, self)
    
    def __mul__(self, other):
        if type(other) in [float, int]:
            return Complex(other * self.real, other * self.imag)
        else:
            return Complex(self.real * other.real - self.imag * other.imag, self.real * other.imag + self.imag * other.real)
        
    def __rmul__(self, other):
        return Complex.__mul__(self, other)
    
    def __str__(self):
        format_str = "{:.%df}{:+.%df}i" % (Complex.print_prec, Complex.print_prec)
        return format_str.format(self.real, self.imag)
    
    def __repr__(self):
        return str(self)

In [ ]:
a = Complex(1,1)
print(abs(a))